In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import re
import collections
import math

In [2]:
rider_names = list(pd.read_csv('../data/pcs-scraping/rider_names.csv'))

In [3]:
rider_name = rider_names[4]
#rider_name = 'julian-alaphilippe'
path = '../data/pcs-scraping/results/rider/'

df_rider = {rider_name: {}}

for file in glob.glob(f'{path}/{rider_name}/*.csv'):
    
    year = re.search('\d{4}', file).group(0)
    df_rider[rider_name][year] = pd.read_csv(file)


In [4]:
rider_name

'egan-bernal'

In [ ]:
#df_2021 = pd.read_csv('./data/pcs-scraping/results/rider//julian-alaphilippe/2015.csv')

# Add Column with RaceType and SubRaceType

In [ ]:
def getMainRaceType(race):
    
    if race.split('(')[-1].startswith('1'):
        
        race_type = 'One Day Race'

    elif race.split('(')[-1].startswith('2'):
        
        race_type = 'Stage Race'
        
    elif race.endswith('WC)'):
        
        race_type = 'One Day Race'
        
    elif race.endswith('NC)'):
        
        race_type = 'One Day Race'
    
    elif race.endswith('CC)'):
        
        race_type = 'One Day Race'
        
    elif race.endswith('Olympics)'):
        
        race_type = 'One Day Race'
    
    elif race.endswith('JR)'):
        
        race_type = 'One Day Race'
    
    else:
        
        return np.nan
    
    return race_type

def getSubRaceType(race):
    
    if '.' in race:
    
        if 'UWT' in race.split('.')[-1].strip(')'):

            race_type = 'UWT'

        elif 'Pro' in race.split('.')[-1].strip(')'):

            race_type = 'Pro'

        elif race.split('.')[-1].strip(')') == '1':

            race_type = '1'

        elif race.split('.')[-1].strip(')') == '2':

            race_type = '2'

        elif 'HC' in race.split('.')[-1].strip(')'):

            race_type = 'HC'

        elif 'NCup' in race.split('.')[-1].strip(')'):

            race_type = 'NCup'

        elif race.split('.')[-1].strip(')')[-1] == 'U':

            race_type = 'U'

        else:

            race_type = np.nan
    
    else:
        
        if 'WC' in race.split('.')[-1].strip(')'):

            race_type = 'WC'

        elif 'CC' in race.split('.')[-1].strip(')'):

            race_type = 'CC'
            
        elif 'JR' in race.split('.')[-1].strip(')'):

            race_type = 'JR'
            
        elif 'Olympics' in race.split('.')[-1].strip(')'):

            race_type = 'Olympics'
            
        elif 'NC' in race.split('.')[-1].strip(')'):

            race_type = 'NC'
        
        else:
            
            race_type = np.nan
            
    return race_type

In [ ]:
for year, df in df_rider[rider_name].items():
    
    race_type = [getMainRaceType(x) for x in df['StageRaceName']]
    df['RaceType'] = race_type
    
    sub_race_type = [getSubRaceType(race) for race in df['StageRaceName']]
    df['SubRaceType'] = sub_race_type

In [ ]:
df_rider[rider_name]['2021'].head(5)

# Add Stage Race UCI Category to RaceType and SubRaceType

In [ ]:
def getRaceTypeStageRace(races):
    
    race = []
    
    for i, x in enumerate(races):
    
        if isinstance(x, str):

            cache = x
            race.append(x)

        elif math.isnan(x):

            race.append(cache)
            
    return race

In [ ]:
for year, df in df_rider[rider_name].items():
    
    main_race_type = getRaceTypeStageRace(df_rider[rider_name][year]['RaceType'])
    df_rider[rider_name][year]['RaceType'] = main_race_type

    
    sub_race_type = getRaceTypeStageRace(df_rider[rider_name][year]['SubRaceType'])
    df_rider[rider_name][year]['SubRaceType'] = sub_race_type

main_race_type = getRaceTypeStageRace(df_rider[rider_name]['2021']['RaceType'])

df_rider[rider_name]['2021']['RaceType'] = main_race_type

sub_race_type = getRaceTypeStageRace(df_rider[rider_name]['2021']['SubRaceType'])

df_rider[rider_name]['2021']['SubRaceType'] = sub_race_type

In [ ]:
df_rider[rider_name]['2021'].head(20)

# Add StageRace Name

In [ ]:
def getStageRaceName(races):
    
    race_name = []
    
    for i, race in races.iterrows():
        
        if race['RaceType'] == 'One Day Race':
            
            race_name.append(race['Race'])
            
        elif race['RaceType'] == 'Stage Race':
            
            if race['Race'].endswith(')') and 'Stage' not in race['Race']:
                
                race_name.append(race['Race'])
            
            else:
                
                race_name.append(race_name[-1])
                
    return race_name

In [ ]:
for year, df in df_rider[rider_name].items():
    
    df_rider[rider_name][year]['StageRaceName'] = getStageRaceName(df_rider[rider_name][year][['Race', 'RaceType']])

df_rider[rider_name]['2021']['StageRaceName'] = getStageRaceName(df_rider[rider_name]['2021'][['Race', 'RaceType']])

In [ ]:
df_rider[rider_name]['2021'].head(15)

# Add GC Results

In [ ]:
def getGCResult(df_rider):

    to_modify = []

    stage_races = df_rider[df_rider['RaceType'] == 'Stage Race']

    for i, race in stage_races.iterrows():

        if 'General classification' in race['Race']:

            to_modify.append((race['StageRaceName'], race['Result']))
          
    for mod in to_modify:
    
        df_rider.loc[df_rider['Race'].str.contains(mod[0], regex=False), 'Result'] = mod[1]    
    
    return df_rider

def getGCPreviousResult(df_rider):
        
    gc = []

    stage_races = df_rider[df_rider['RaceType'] == 'Stage Race']

    for stage_race in stage_races.groupby('StageRaceName'):

        ind = stage_race[1][stage_race[1]['Race'].str.contains('Stage')].index[0]
        val = stage_race[1].loc[ind+1, 'GC']
        stage_race[1].loc[ind, 'GC'] = val
        
        df_rider.loc[df_rider['StageRaceName']==stage_race[0], 'GC'] = stage_race[1]['GC']
        
    return df_rider

In [ ]:
for year, df in df_rider[rider_name].items():
    
    df_rider[rider_name][year] = getGCResult(df_rider[rider_name][year])
    df_rider[rider_name][year] = getGCPreviousResult(df_rider[rider_name][year])

df_rider[rider_name]['2021'] = getGCResult(df_rider[rider_name]['2021'])
df_rider[rider_name]['2021'] = getGCPreviousResult(df_rider[rider_name]['2021'])

# Prep Data for Plotting

In [ ]:
def normalizeDate(date):
    
    if not math.isnan(date):
        
        date = str(date)
        
        if len(date.split('.')[-1]) == 1:
            
            date = date + '0'
            
        if len(date.split('.')[0]) == 1:
            
            date = '0' + date
            
    return date

def prepData(df_rider, rm_non_numeric=False):
    
    df_rider_clean = df_rider.copy()

    for year, df in df_rider_clean[rider_name].items():

        df_dict = df.copy()

        # remove NaN values
        df_dict.dropna(subset=['Result'], inplace=True)
        df_dict.dropna(subset=['Date'], inplace=True)
        
        # dates to timestamp
        df_dict['Date'] = df_dict['Date'].apply(lambda x: normalizeDate(x))
        df_dict['Date'] = df_dict['Date'].apply(lambda x: str(x)+f'.{year}')
        df_dict['Date'] = df_dict['Date'].apply(lambda x: pd.to_datetime(x, format='%d.%m.%Y'))
        
        # remove string values
        if rm_non_numeric:
            df_dict = df_dict[(df_dict.Result != 'DNS') & (df_dict.Result != 'DNF') & (df_dict.Result != 'DF')]

        # define Result as int
        to_exlude = ['DNF', 'DNS', 'DF']
        results = [int(x) if x not in to_exlude else x for x in df_dict['Result']]
        gc = [int(x) if not math.isnan(x) else np.nan for x in df_dict['GC']]
        df_dict['Result'] = results
        df_dict['GC'] = gc

        df_rider_clean[rider_name][year] = df_dict.sort_values(by='Date')
        
    return df_rider_clean

def getNumericData():
    
    pass

In [ ]:
df_rider_clean = prepData(df_rider, rm_non_numeric=True)

In [ ]:
for year, df in df_rider_clean[rider_name].items():
    
    df.to_csv(f'{path}/{rider_name}/{year}_clean.csv', index=False)

- progression of top_x over seasons
- stage difficulty index (race ranking scraped on race link, vert. meters)
- model: X (Date, current GC, dist, vert, uci points, racetype, subracetype, racename, weather?), y (Result)
- strategy for teams to win points and stay/progress to world tour ?
- get 'won how' data and define strategy based on stage ?
- when this info is missing, impute sprint if same arrival time, solo if time gaps, etc.
- junior/u23 rider potential: clustering of results compared to successful riders
- world tour teams performance analysis -> race selection optimization